# How to Search for Catalog Data and IDs with lksearch

`lksearch` enables you to search through the catalogs associated with each mission, and Gaia. You can query:

- The Kepler Input Catalog
- The Ecliptic Plane Input Catalog
- The TESS Input Catalog
- Gaia EDR3.

`lksearch` has three main pieces of functionality - two ways of retrieving data from a catalog, and one for looking up catalog IDs.

These are:
   
- **A position cone-search** that searches for sources within a given radius from some position using `QueryPosition`
- **An exact ID lookup**, retrieving catalog rows where the catalog ID matches the search ID using `QueryID`
- **A way of looking up catalog IDs** given a common name or another catalog ID with `IDLookup`. 

`lksearch` also has the ability to crossmatch targets when doing an exact ID lookup between the TIC, KIC, and Gaia EDR3 catalogs because this crossmatch already exists in the TIC.  

`lksearch` will propagate the proper motion for you, and can optionally return a SkyCoordinate either directly from the Search or by using the `to_skycoord` property from the results table.  

Below are examples of each query.

## Searching by Position

In [1]:
from lksearch.catalogsearch import (
    query_region,
    query_id,
    query_names,
    match_names_catalogs,
)
from astropy.coordinates import SkyCoord
from astropy.time import Time
import astropy.units as u

Before we search we need a sky position to search. `lksearch` catalog queries use `astropy`'s `SkyCoord` as an input. We can either make one ourselves with an RA and Dec, or we can use the built in `from_name` method.

In [2]:
c = SkyCoord.from_name("Kepler-10")

Now we have a position we want to query, we can query each catalog. For example, here is the query for KIC

In [3]:
query_region(c, catalog="kic", max_results=5)

,ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,Kepmag,Teff,logg
0,KIC 11904151,285.67938,50.241299,0.000000,1.000000,0.0,0.0,10.961000,5491,4.467
1,KIC 11904160,285.68076,50.242493,5.345214,0.006644,0.0,0.0,16.405001,<NA>,NaN
2,KIC 11904158,285.68042,50.245716,16.080462,0.005420,0.0,0.0,16.625999,<NA>,NaN
3,KIC 11904156,285.68015,50.250011,31.413257,0.002672,-4.0,0.0,17.393999,<NA>,NaN
4,KIC 11904148,285.67722,50.259010,63.953186,0.017155,0.0,0.0,15.375000,5286,4.640


By default, the catalog will query with an epoch of that is the default for the catalog. You can set the `epoch` keyword to a date in order to get `RA` and `Dec` at that date, given the proper motion.

In [4]:
query_region(c, output_epoch=Time.now(), catalog="kic", max_results=5)

,ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,Kepmag,Teff,logg
0,KIC 11904151,285.679380,50.241299,0.000000,1.000000,0.0,0.0,10.961000,5491,4.467
1,KIC 11904160,285.680760,50.242493,5.345214,0.006644,0.0,0.0,16.405001,<NA>,NaN
2,KIC 11904158,285.680420,50.245716,16.080462,0.005420,0.0,0.0,16.625999,<NA>,NaN
3,KIC 11904156,285.680107,50.250011,31.407788,0.002672,-4.0,0.0,17.393999,<NA>,NaN
4,KIC 11904148,285.677220,50.259010,63.953186,0.017155,0.0,0.0,15.375000,5286,4.640


You can also pass a radius for your query. If you query each catalog directly you can set your query radius in units of pixels, otherwise you can use arcseconds.

In [5]:
query_region(c, output_epoch=Time.now(), radius=30 * u.arcsec, catalog="kic")

,ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,Kepmag,Teff,logg
0,KIC 11904151,285.67938,50.241299,0.000000,1.000000,0.0,0.0,10.961000,5491,4.467
1,KIC 11904160,285.68076,50.242493,5.345214,0.006644,0.0,0.0,16.405001,<NA>,NaN
2,KIC 11904165,285.68317,50.241207,8.732390,0.002660,0.0,0.0,17.399000,<NA>,NaN
3,KIC 11904162,285.68124,50.243916,10.348793,0.001576,0.0,0.0,17.966999,<NA>,NaN
4,KIC 11904158,285.68042,50.245716,16.080462,0.005420,0.0,0.0,16.625999,<NA>,NaN


You can query TIC

In [6]:
query_region(c, output_epoch=Time.now(), radius=20 * u.arcsec, catalog="tic")

,ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,TESSmag,Mass,Rad,Teff,logg
0,TIC 377780790,285.679223,50.241593,NaN,915.377136,-18.394,41.448,10.477000,1.017,1.09,5706.0,4.3707
1,TIC 1717218059,NaN,NaN,NaN,1.000000,-2.227,-1.204,17.881001,1.070,0.81,5895.0,4.6506


EPIC (Note Kepler-10 isn't inside EPIC, because it's outside the ecliptic plane!)

In [7]:
query_region(c, catalog="epic")

,RAJ2000,DEJ2000,pmRA,pmDE,Plx,K2mag,logg,Teff,Rad,Mass,RA,Dec,Separation,Relative_Flux
ID,,,,,,,,,,,,,,


And Gaia in the same ways

In [8]:
query_region(c, output_epoch=Time.now(), radius=40 * u.arcsecond, catalog="gaiadr3")

,ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,BPmag,Gmag,RPmag,Teff,logg
0,Gaia DR3 2132155017099178624,285.679351,50.241409,0.000000,1.000000,-18.483,41.382,11.253505,10.920031,10.418263,5595.8,4.3544
1,Gaia DR3 2132155051458917632,285.680213,50.249942,30.783367,0.001691,-1.519,-1.203,18.209782,17.849485,17.317846,5424.7,4.6774
2,Gaia DR3 2132155051458918144,285.685923,50.249919,34.169635,0.005578,9.402,5.814,17.213984,16.553818,15.778805,NaN,NaN


Optionally, you can return the output of the search as an `astropy.coordinates.SkyCoord` object

In [9]:
query_region(
    c,
    output_epoch=Time.now(),
    radius=40 * u.arcsecond,
    catalog="gaiadr3",
    return_skycoord=True,
)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(285.67935079, 50.24140863,  186.22667508),
     (285.68021336, 50.24994175, 7132.66761762),
     (285.68592306, 50.24991892, 1124.73287589)]
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    [(-18.48303979, 41.38198223, 7.84849083e-05),
     ( -1.5189999 , -1.20300012, 5.49460825e-06),
     (  9.40200284,  5.8139954 , 2.81997883e-05)]>

## Searching by Catalog ID

For TIC, KIC, EPIC, and Gaia DR3 - if we know the Catalog ID of a source we can use `QueryID` to return the data from that catalog for that id.  For example, Kepler-10 above is also known as TIC 377780790. 

In [10]:
query_id("TIC 377780790")

,TIC,RAJ2000,DEJ2000,HIP,TYC,UCAC4,_2MASS,objID,WISEA,GAIA,...,Clist,e_RAJ2000,e_DEJ2000,RAOdeg,DEOdeg,e_RAOdeg,e_DEOdeg,RadFl,WDFl,ID
0,377780790,285.679422,50.241306,<NA>,,702-062695,19024305+5014286,<NA>,J190243.03+501429.1,2132155017099178624,...,,0.767293,0.711508,285.679298,50.241484,0.020474,0.024043,1,0,237087497


... and KIC 11904151

In [11]:
query_id("KIC 11904151")

,KIC,RAJ2000,DEJ2000,pmRA,pmDE,Plx,umag,gmag,rmag,imag,...,cq,pq,aq,fv,Teff,logg,__Fe_H_,E_B-V_,Av,R_
0,11904151,285.67938,50.241299,0.0,0.0,NaN,NaN,11.388,10.92,10.778,...,SCP,5,6,2,5491,4.467,-0.812,0.031,0.095,0.98


... and Gaia DR3 2132155017099178624

In [12]:
query_id("Gaia DR3 2132155017099178624")

,DR3Name,RA_ICRS,DE_ICRS,SolID,Source,RandomI,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,...,RAVE6,dRAVE6,f_RAVE6,RAVE6oid,nRAVE6,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
0,Gaia DR3 2132155017099178624,285.679294,50.24149,1636148068921376768,2132155017099178624,712720541,0.0097,0.0105,5.3698,0.0103,...,,NaN,<NA>,NaN,<NA>,285.679422,50.241306,0.199479,0.217694,-0.0523


The TIC catalog has been cross-matched with the GAIA DR3 and KIC catalogs.  `lksearch` can use this cross-matching to retrieve TIC/KIC/GAIA catalog info for any given TIC/KIC/GaiaDR3 source ID.  The EPIC catalog is not included in this cross-matching.  

For Example, if we want the TIC info for Kepler-10, but only have the GAIA DR3Name above, we can get that like so:

In [13]:
query_id("Gaia DR3 2132155017099178624", output_catalog="tic")

,TIC,RAJ2000,DEJ2000,HIP,TYC,UCAC4,_2MASS,objID,WISEA,GAIA,...,Clist,e_RAJ2000,e_DEJ2000,RAOdeg,DEOdeg,e_RAOdeg,e_DEOdeg,RadFl,WDFl,ID
0,377780790,285.679422,50.241306,<NA>,,702-062695,19024305+5014286,<NA>,J190243.03+501429.1,2132155017099178624,...,,0.767293,0.711508,285.679298,50.241484,0.020474,0.024043,1,0,237087497


We can also supply an integer ID, but if so we MUST specify what catalog to search

In [14]:
query_id("2132155017099178624", output_catalog="gaiadr3")

,DR3Name,RA_ICRS,DE_ICRS,SolID,Source,RandomI,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,...,RAVE6,dRAVE6,f_RAVE6,RAVE6oid,nRAVE6,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
0,Gaia DR3 2132155017099178624,285.679294,50.24149,1636148068921376768,2132155017099178624,712720541,0.0097,0.0105,5.3698,0.0103,...,,NaN,<NA>,NaN,<NA>,285.679422,50.241306,0.199479,0.217694,-0.0523


If we specify an integer and wish to use our limited cross-match, we can do that by specifying both the `input_catalog` and `catalog` parameters

In [15]:
query_id("2132155017099178624", output_catalog="tic", input_catalog="gaiadr3")

,TIC,RAJ2000,DEJ2000,HIP,TYC,UCAC4,_2MASS,objID,WISEA,GAIA,...,Clist,e_RAJ2000,e_DEJ2000,RAOdeg,DEOdeg,e_RAOdeg,e_DEOdeg,RadFl,WDFl,ID
0,377780790,285.679422,50.241306,<NA>,,702-062695,19024305+5014286,<NA>,J190243.03+501429.1,2132155017099178624,...,,0.767293,0.711508,285.679298,50.241484,0.020474,0.024043,1,0,237087497


We can also search for multiple IDS from a single catalog and get a table of all of the resulting catalog entries

In [16]:
# Kepler Objects 1-10
idlist = [
    "TIC 399860444",
    "TIC 424865156",
    "TIC 28230919",
    "TIC 350814622",
    "TIC 268823307",
    "TIC 27916356",
    "TIC 121660904",
    "TIC 123233041",
    "TIC 120571842",
    "TIC 377780790",
]
query_id(idlist, output_catalog="tic")

,TIC,RAJ2000,DEJ2000,HIP,TYC,UCAC4,_2MASS,objID,WISEA,GAIA,...,Clist,e_RAJ2000,e_DEJ2000,RAOdeg,DEOdeg,e_RAOdeg,e_DEOdeg,RadFl,WDFl,ID
0,27916356,296.837241,48.239933,<NA>,,692-068460,19472094+4814238,<NA>,J194720.94+481423.9,2086636884980514304,...,,0.431661,0.401100,296.837248,48.239969,0.011393,0.014386,1,0,338972619
1,28230919,297.709364,48.080860,97657,3561-02092-1,691-070316,19505021+4804508,<NA>,,2086512227851023872,...,,0.689515,0.697757,297.710176,48.081864,0.019271,0.024141,1,0,347873767
2,120571842,285.573977,38.400883,<NA>,,643-065464,19021775+3824032,1237668734687445760,J190217.77+382402.9,2099925719951103872,...,,0.460796,0.495197,285.573990,38.400819,0.013008,0.016279,1,0,236288245
3,121660904,288.581510,41.089824,<NA>,,656-070649,19141956+4105233,<NA>,J191419.55+410523.1,2102117871259036672,...,,0.830982,0.646732,288.581493,41.089734,0.021624,0.020590,1,0,259926978
4,123233041,281.288121,42.451081,<NA>,,663-070081,18450914+4227038,<NA>,J184509.14+422703.9,2116730994965905280,...,,0.388089,0.420084,281.288134,42.451098,0.011065,0.014161,1,0,214866427
5,268823307,299.407035,44.035053,<NA>,,671-080900,19573768+4402061,<NA>,J195737.69+440206.0,2079018300195390464,...,,0.605437,0.487757,299.407034,44.035039,0.018613,0.016314,1,0,363112972
6,350814622,285.615409,50.135751,<NA>,,701-064857,19022767+5008087,<NA>,J190227.69+500808.7,2132152916856093952,...,,0.670902,0.662993,285.615449,50.135771,0.019169,0.024925,1,0,237044605
7,377780790,285.679422,50.241306,<NA>,,702-062695,19024305+5014286,<NA>,J190243.03+501429.1,2132155017099178624,...,,0.767293,0.711508,285.679298,50.241484,0.020474,0.024043,1,0,237087497
8,399860444,286.808491,49.316414,<NA>,3549-02811-1,697-064165,19071403+4918590,<NA>,J190714.04+491859.0,2131314401800665344,...,,0.734770,0.672364,286.808526,49.316421,0.019726,0.022829,1,0,245833148
9,424865156,292.247308,47.969505,<NA>,3547-01402-1,690-068273,19285935+4758102,<NA>,J192859.33+475810.3,2129256395211984000,...,,0.645983,0.573522,292.247190,47.969543,0.017649,0.019830,1,0,290881018


Large queries (over one thousand objects) will be performed asynchronously and may take some time.  

Like `QueryPosition`, These results can also be returned as `astropy SkyCoord` objects.  

In [17]:
query_id("TIC 377780790", return_skycoord=True)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(285.67942179, 50.24130577, 186.50105373)]
 (pm_ra_cosdec, pm_dec) in mas / yr
    [(-18.394, 41.448)]>

## Looking up Catalog IDs

We have also included a limited way of looking up catalog IDs using other known IDs.  This functionality wraps the `astroquery.simbad.Simbad.query_objectids` functionality.  

In [18]:
query_names("Kepler 10")

,id
0,LAMOST J190243.11+501428.7
1,AP J19024305+5014286
2,Gaia DR3 2132155017099178624
3,TIC 377780790
4,SPOCS 4268
...,...
9,PPMX J190243.0+501428
10,USNO-B1.0 1402-00324696
11,GSC 03549-00354
12,KOI-72


It is based off of joining IDs across Simbad tables, and is useful but not comprehensive as it relies upon published works to do the ID crossmatching.  That means that you may run into situations where not all assosciations have been made, eg. for this EPIC ID:

In [19]:
query_names("EPIC 201563164")

,id
0,Gaia DR3 3796414192429498880
1,LBQS 1145+0145
2,WD 1145+017
3,SDSS J114833.62+012859.4
4,EPIC 201563164
5,HE 1145+0145
6,Gaia DR2 3796414192429498880


We can see above that this EPIC ID has a Gaia DR3 ID assosciated with it, and we know that GAIA DR3 and TIC are crossmatched, but the TIC ID is not reported.  To retrieve the TIC ID we can use the GAIA DR3 ID above and QueryID

In [20]:
query_names("Gaia DR3 3796414192429498880")

,id
0,Gaia DR3 3796414192429498880
1,LBQS 1145+0145
2,WD 1145+017
3,SDSS J114833.62+012859.4
4,EPIC 201563164
5,HE 1145+0145
6,Gaia DR2 3796414192429498880


`lksearch` also alows you to input a list of strings, which will then perform sequential ID searches for each item in the list and return a list of the resulting tables: 

In [21]:
query_names(["Kepler 10", "Kepler 16"])

Throttling query limit to Simbad's: max 5/s


[                              id
 0     LAMOST J190243.11+501428.7
 1           AP J19024305+5014286
 2   Gaia DR3 2132155017099178624
 3                  TIC 377780790
 4                     SPOCS 4268
 ..                           ...
 9          PPMX J190243.0+501428
 10       USNO-B1.0 1402-00324696
 11               GSC 03549-00354
 12                        KOI-72
 13                  KIC 11904151
 
 [14 rows x 1 columns],
                               id
 0     LAMOST J191618.16+514526.7
 1   Gaia DR3 2133476355197071616
 2                  TIC 299096355
 3   Gaia DR2 2133476355197071616
 4        2MASS J19161817+5145267
 ..                           ...
 6                GSC 03554-01147
 7                   KIC 12644769
 8                      Kepler-16
 9            NAME Kepler-16 (AB)
 10                      KOI-1611
 
 [11 rows x 1 columns]]

### Matching ID Results to Catalogs
`lksearch` has the ability to parse the id lists returned by `QueryID` to look for ids which contain the string contained in the optional parameter `match`.  This can be usefull for only returning the catalog results which interest you. 

Just like the search input can be a list, so can the match criteria, as so: 

In [22]:
match_names_catalogs(["Kepler 10", "Kepler 16"], match=["tic", "kic"])

Throttling query limit to Simbad's: max 5/s


,search,tic,kic
0,Kepler 10,TIC 377780790,KIC 11904151
1,Kepler 16,TIC 299096355,KIC 12644769
